<a href="https://colab.research.google.com/github/Imperial-lord/sdn-controllers-load-balancing/blob/main/SDN_Load_Balancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
####### Graph Building using data from Arnes.gml ########
import networkx as nx
import time
import random
import math
import numpy as np
import pandas as pd

from math import radians, cos, sin, asin, sqrt

####### Data collection, cleaning, and handling corner cases #######

k = int(input("Enter the number of controllers to allocate: "))
start = time.time()
graph = nx.read_gml('/content/drive/MyDrive/BTP Documents/Arnes.gml', label='id')

lat = nx.get_node_attributes(graph, "Latitude")
lon = nx.get_node_attributes(graph, "Longitude")

# Remove bad nodes from 'graph'
for node in list(graph.nodes):
    # removing nodes 
    # (a) with undefined latitude or longitude  
    # (b) that are isolated
    check_bad_node = lat.get(node) == None or lon.get(node) == None or nx.is_isolate(graph,node)
    if check_bad_node == True:
        graph.remove_node(node)


def dist_between_nodes(node_1, node_2):
    '''Compute distance between any 2 nodes of 'graph' using Haversine formula

    Args:
        node_1 (int): The first among the 2 graph nodes 
        node_2 (int): The second among the 2 graph nodes 

    Returns:
        dist: distanct between the two nodes
    '''
    lat_1 = radians(lat.get(node_1))
    lat_2 = radians(lat.get(node_2))
    lon_1 = radians(lon.get(node_1))
    lon_2 = radians(lon.get(node_2))

    # Applying Haversine formula
    dlon = lon_2 - lon_1
    dlat = lat_2 - lat_1
    a = sin(dlat/2)**2 + cos(lat_1)*cos(lat_2)*sin(dlon/2)**2
    c = 2*asin(sqrt(a))

    # Mean radius of Earth
    R = 6371.009
    dist = c*R

    return dist

# Assigning weights (based on distance) to the edges of the graph
for node in list(graph.nodes):
    for neighbor in list(graph.neighbors(node)):
        graph[node][neighbor]['weight'] = dist_between_nodes(node, neighbor)

# Removing edges with no weights assigned
for edge in list(graph.edges):
    if 'weight' not in graph.edges[edge] == False:
        graph.remove_edge(edge[0],edge[1])

print(nx.info(graph))
n = nx.number_of_nodes(graph)

####### Graph building and Controllers' Cluster formation #######

# Store the graph as an adjacency list
adj_list = [];
for i in range(0,n):
    adj_list.append([])

for edge in graph.edges:
    i,j = edge
    adj_list[i].append(j)
    adj_list[j].append(i)

# Random sampling for controller index selection
controller = random.sample(range(0,n),k)

# List of lists represents index of switches in particular controller 
# e.g. [[s1,s2],[s3,s4],[s5]] controller distribution set
lol = []
for i in range(0,k):
    lol.append([])

# Controllers set
controller_max = list(controller)
print("\nControllers: \n{}\n".format(controller))

def BFS(queue, adj_list, listx):
    '''Performs a breadth first search in the graph
    '''
    while queue:
        p = queue.pop(0)
        u, w = p
        for v in adj_list[u]:
            if listx[v] == math.inf:
                listx[v] = w + 1
                queue.append((v, w + 1))
    return listx

index = 0
for j in range(0, k):
    listx = n*[math.inf]
    listx[controller[index]] = 0
    queue = []
    queue.append((controller[index], 0))
    listx = BFS(queue, adj_list, listx)
    lol[j] = listx
    index += 1

transpose_list = np.transpose(lol).tolist()
minumum_pos = []

for x in transpose_list:
    min_ele = n+5
    for i in range(0, len(x)):
        if x[i] < min_ele:
            min_ele = x[i]
    ts = []
    for i in range(0, len(x)):
        if(x[i] == min_ele):
            ts.append(i)
    minumum_pos.append(random.choice(ts))

final = []
controller_set = []
for i in range(0, k):
    controller_set.append([])

for i in range(0, n):
    final.append((i, controller[minumum_pos[i]]))
    controller_set[minumum_pos[i]].append(i)

print('Cluster set for each controller: \n{}\n'.format(controller_set))


# Keeping another copy of the controller set for Q-learning
controller_set_max = list(controller_set)

frame = pd.read_csv('/content/drive/MyDrive/BTP Documents/data.csv')
nodes = list(graph.nodes)
random.shuffle(nodes)
frame = frame.rename(columns=dict(zip(frame.columns, nodes)))

load_array = []
for i in frame.index:
    Load = dict(frame.loc[i])
    load_array.append(Load)
    

Enter the number of controllers to allocate: 12
Graph with 34 nodes and 46 edges

Controllers: 
[8, 20, 12, 0, 31, 9, 10, 2, 11, 22, 18, 29]

Cluster set for each controller: 
[[5, 8], [6, 15, 20], [12, 26], [0, 7], [31], [9, 19], [10, 17], [2, 3, 21, 24, 25, 32, 33], [11], [4, 22, 23], [14, 16, 18], [1, 13, 27, 28, 29, 30]]



In [47]:
####### Common functions and calculations for controller load #######

# Classify the domain based on load as overloaded or underloaded
def classify_domain(cluster_load, no_of_controller):
    # Mean load
    mean_load = sum([y for x, y in cluster_load.items()])
    mean_load = mean_load/no_of_controller
    O_domain = []
    I_domain = []
    for controller1, controller_load in cluster_load.items():
        if(controller_load > mean_load):
            O_domain.append(controller1)
        else:
            I_domain.append(controller1)
    return [O_domain, I_domain, mean_load]


# Update the load
def update_load(cluster_load, i, current_state, cnt, current_switch):
    cluster_load[current_state] = cluster_load[current_state] - \
        load_array[i][current_switch]
    cluster_load[cnt] = cluster_load[cnt]+load_array[i][current_switch]

# Return all available actions in state given as an argument
def available_actions(k):
    return [item for item in range(0, k)]

# Calculate the mean load
def load_rate(cluster_load, mean_load, no_of_controller):
    s = 0
    for controller1, controller_load in cluster_load.items():
        s = s + abs(controller_load - mean_load)
    s = s/no_of_controller
    return s

# Calculate load ratio
def load_ratio_calculate(controller1, cluster_load):
    return cluster_load[controller1]


In [ ]:
####### Defining functions for population of cluster load array 
####### switch selection, migration, and random actions ####### 

# Populating cluster load array
# {c1:36363663, c2:3234325, c3:32523525, c4:3523535, c5:3542646}
def calculate_load(time_step):
    cluster_load = defaultdict(lambda: 0)
    for center in controller:
        # print(center)
        for node in controller_set:
            # print(node)
            if center in node:
                for switch in node:
                    # print(switch)
                    cluster_load[center] += load_array[time_step][switch]
    return cluster_load

# For switch selection randomly
def switch_select(ind, current_cont):
    if(len(controller_set[ind]) == 1):
        return -1
    x = random.choice(controller_set[ind])
    # To avoid controller to be selected itself
    while(x == current_cont):
        x = random.choice(controller_set[ind])
    return x

# Random controller selection
def random_action(available_act, control):
    next_action = int(np.random.choice(available_act, 1))
    while(next_action == control):
        next_action = int(np.random.choice(available_act, 1))
    return next_action

# Migration of switch from source to target
def migrate_switch(ind_of_controller, target_controller, current_switch):
    controller_set[ind_of_controller].remove(current_switch)
    controller_set[target_controller].append(current_switch)

# Selection of switch
def switch_select_mx(ind, current_cont):
    if(len(controller_set[ind]) == 1):
        return -1
    x = random.choice(controller_set[ind])
    while(x == current_cont):
        x = random.choice(controller_set[ind])
    return x